# Set Up

In [ ]:
from IPython.display import clear_output 
!pip install tensorflow==2.9 t5 tensorflow-text==2.9
#!pip install -q t5 tensorflow-text==2.4.3
#!pip install -q tensorflow-text==2.8.0rc0
#!pip install -U tensorflow-gcs-config==2.9.1
clear_output()

In [ ]:
print("Installing dependencies...")
import functools
import os
import gin
import tensorflow_gcs_config
from google.colab import auth
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
from contextlib import contextmanager
import logging as py_logging
import t5

Installing dependencies...


In [ ]:
TOKENIZER_DIR = "bucket path" #@param { type: "string" }
if not TOKENIZER_DIR or TOKENIZER_DIR == "gs://": 
  raise ValueError("You must enter a TOKENIZER_DIR.")

print("Setting up GCS access...")
os.environ['USE_AUTH_EPHEM'] = '0'
from google.colab import auth
auth.authenticate_user()

# Set credentials for GCS reading/writing from Colab and TPU.
TPU_TOPOLOGY = "2x2"
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  TPU_ADDRESS = tpu.get_master()
  print('Running on TPU:', TPU_ADDRESS)
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
tf.config.experimental_connect_to_host(TPU_ADDRESS)
tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()


#LOGGING
tf.get_logger().propagate = False
py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Setting up GCS access...


Instructions for updating:
non-resource variables are not supported in the long term


Running on TPU: grpc://10.72.61.82:8470


# Load Vocabulary

In [ ]:
vocab_model_path = 'gs://bucket_context/eighth_experiment/code.model'
vocab_path = 'gs://bucket_context/eighth_experiment/code.vocab'
print(vocab_model_path)
print(vocab_path)

gs://bucket_context/eighth_experiment/code.model
gs://bucket_context/eighth_experiment/code.vocab


In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary

num_special_mask_tokens = 100 #@param {type: "integer"}

def load_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, num_special_mask_tokens)

In [ ]:
# change config file based on the finetuning context you want to perform
config="call"

# Prepare Dataset for T5

In [ ]:
# save each dataset in a folder named ft_ followed by the current context
train_path = 'gs://bucket_context/eighth_experiment/ft_{}/train.tsv'.format(config) #@param { type: "string" }
eval_path = 'gs://bucket_context/eighth_experiment/ft_{}/eval.tsv'.format(config) #@param { type: "string" }
test_path = 'gs://bucket_context/eighth_experiment/ft_{}/test.tsv'.format(config) #@param { type: "string" }
finetune_datasets_paths = {
    "train":      train_path,
    "validation": test_path
}

# Useful when multi-task training 
# num_input_examples = dict(train=106382, validation=12020) 

In [ ]:
def load_dataset(split, shuffle_files=True):
  """
  Function to load .tsv dataset as a tf.data.Dataset in TensorFlow
  """
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(finetune_datasets_paths[split])
  ds = ds.map(functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                          field_delim="\t", use_quote_delim=False)
                          , 
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

### A few examples

In [ ]:
print("A few raw validation examples...")
for ex in tfds.as_numpy(load_dataset("validation").take(5)):
  print(ex)

A few raw validation examples...
{'input': b"void processError(String errorText) {<nl>this.statistics.incrementErrCount();<nl>this.lastError.set(errorText);<nl>this.connectError.set(errorText); // even if this isn't during connection, save it just in case<nl>// If we are connected && we get an authentication error, save it<nl>String url = this.getConnectedUrl();<nl>if (this.isConnected() && this.isAuthenticationError(errorText) && url != null) {<nl>this.serverAuthErrors.put(url, errorText);<nl>}<nl><extra_id_0><nl>try {<nl>this.callbackRunner.execute(() -> {<nl>try {<nl>options.getErrorListener().errorOccurred(this, errorText);<nl>} catch (Exception ex) {<nl>this.statistics.incrementExceptionCount();<nl>}<nl>});<nl>} catch (RejectedExecutionException re) {<nl>// Timing with shutdown, let it go<nl>}<nl>}<nl>} <sep> public void setTcpNoDelay(boolean on) {<nl>this.tcpNoDelay = on;<nl>if (this.session != null && this.session instanceof TCPSession) {<nl>try {<nl>((SocketChannel) ((TCPSessio

# Dataset Prepocessing 

In [ ]:
from tensorflow_datasets.core.utils.type_utils import Shape

def preprocessing(ds):
  """
  Preprocess function to convert the tf.data.Dataset into a text-to-text format,
  with both inputs and targets fields.
  Param: tf.data.Dataset
  Return: text-to-text format
  """
  prefix = '' # no prefix for pretraining
  def to_inputs_and_targets(ex):
    x_input = tf.strings.strip(prefix + ex['input'])
    y_label = tf.strings.strip(ex['output']) 
    inputs = tf.strings.join([x_input], separator=' ')
    class_label = tf.strings.join([y_label], separator=' ')
    return {'inputs': inputs, 'targets': class_label}
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

### A few examples

In [ ]:
print("A few preprocessed train examples...")
sample = tfds.as_numpy(preprocessing(load_dataset("train").take(5)))
for ex in sample:
  print(ex)

A few preprocessed train examples...
{'inputs': b'<extra_id_0><nl>return MapHandlerRegistration.addHandler(this, MapEventType.SHADOW_CHANGED, handler,<nl>new ShadowChangeEventFormatter());<nl>} <sep> public final HandlerRegistration addAnimationChangeHandler(AnimationChangeMapHandler handler) {<nl>return MapHandlerRegistration.addHandler(this, MapEventType.ANIMATION_CHANGED, handler,<nl>new AnimationChangeEventFormatter());<nl>}', 'targets': b'public final HandlerRegistration addShadowChangeHandler(ShadowChangeMapHandler handler) {'}
{'inputs': b'public static com.oracle.bmc.http.internal.WrappedInvocationBuilder fromRequest(<nl>com.oracle.bmc.http.internal.RestClient client,<nl>com.oracle.bmc.core.requests.UpdateVolumeGroupBackupRequest request) {<nl>Validate.notNull(request, "request instance is required");<nl>Validate.notBlank(<nl>request.getVolumeGroupBackupId(), "volumeGroupBackupId must not be blank");<nl>Validate.notNull(<nl>request.getUpdateVolumeGroupBackupDetails(),<nl>"updat

# Creating Task and Mixture

In [ ]:
DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=load_vocabulary(), add_eos=True, required=False),
    "targets": Feature(
        vocabulary=load_vocabulary(), add_eos=True)
    }

TASK_NAME = "ft" #@param{ type : "string"}

# TASK
t5.data.TaskRegistry.remove(TASK_NAME)
t5.data.TaskRegistry.add(
    TASK_NAME,
    # Function which returns a tf.data.Dataset
    dataset_fn=load_dataset,
    splits=["train","validation"],
    # List of functions that preprocess the input tf.data.Dataset
    text_preprocessor=[preprocessing],
    # Accuracy is used as evaluation metric
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, helps for mixing and auto-caching
    # num_input_examples=num_input_examples,
    output_features = DEFAULT_OUTPUT_FEATURES
)

MIXTURE_NAME = "task" #@param{ type : "string"}

# MIXTURE
t5.data.MixtureRegistry.remove(MIXTURE_NAME)
t5.data.MixtureRegistry.add(
    MIXTURE_NAME,
    # List of tasks
    [TASK_NAME],
    default_rate=1.0
)


### A few examples

In [ ]:
finetuning_task = t5.data.TaskRegistry.get(TASK_NAME)
ds = finetuning_task.get_dataset(split="train", sequence_length={"inputs": 1024, "targets": 1024})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

A few preprocessed training examples...
{'inputs_pretokenized': b'protected Map<String, Map<String, Set<String>>> scan() {<nl>long start = System.currentTimeMillis();<nl>Map<String, Set<Map.Entry<String, String>>> collect = configuration.getScanners().stream().map(Scanner::index).distinct()<nl>.collect(Collectors.toMap(s -> s, s -> Collections.synchronizedSet(new HashSet<>())));<nl><extra_id_0><nl>Vfs.Dir dir = null;<nl>try {<nl>dir = Vfs.fromURL(url);<nl>for (Vfs.File file : dir.getFiles()) {<nl>if (doFilter(file, configuration.getInputsFilter())) {<nl>ClassFile classFile = null;<nl>for (Scanner scanner : configuration.getScanners()) {<nl>try {<nl>if (doFilter(file, scanner::acceptsInput)) {<nl>List<Map.Entry<String, String>> entries = scanner.scan(file);<nl>if (entries == null) {<nl>if (classFile == null) classFile = getClassFile(file);<nl>entries = scanner.scan(classFile);<nl>}<nl>if (entries != null) collect.get(scanner.index()).addAll(entries);<nl>}<nl>} catch (Exception e) {<nl>i

# Creating Model

In [ ]:
from t5 import models

FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string ('f', '', 'kernel')

#See https://github.com/google-research/text-to-text-transfer-transformer if you want to scale up the model
MODEL_SIZE = "base"  

MODEL_DIR = 'gs://bucket_context/eighth_experiment/ft_final_model_{}'.format(config)

PRETRAINED_DIR='gs://bucket_context/eighth_experiment/pt_model'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 64, 50),
    "base": (2, 32, 100),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]


tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 1024, "targets": 1024},
    learning_rate_schedule = 0.001,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max
)

In [ ]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = 512
model.eval(
    mixture_or_task_name="task",
    # mixture_or_task_name="all_tasks",
    checkpoint_steps=-1 #evaluate only last checkpoint
)

INFO:root:system_path_file_exists:gs://bucket_context/eighth_experiment/ft_final_model_most_similar_crystalbleu/operative_config.gin
ERROR:root:Path not found: gs://bucket_context/eighth_experiment/ft_final_model_most_similar_crystalbleu/operative_config.gin
INFO:absl:Adding task 'ft' with predict metric_fn(s).
INFO:absl:Skipping packing/padding for 'ft' since sequence length is None.
INFO:absl:Setting sequence lengths to {'inputs': 1345, 'targets': 179}
INFO:absl:Evaluating checkpoint step: 1670000
INFO:absl:Padding 'ft' with sequence lengths: {'inputs': 1345, 'targets': 179}
SimdMeshImpl ignoring devices ['', '', '', '', '', '', '', '']
Using default tf glorot_uniform_initializer for variable encoder/block_000/layer_000/SelfAttention/relative_attention_bias  The initialzer will guess the input and output dimensions  based on dimension order.
Using default tf glorot_uniform_initializer for variable decoder/block_000/layer_000/SelfAttention/relative_attention_bias  The initialzer will 

KeyboardInterrupt: ignored

In [ ]:
# set the best checkpoint for the current context
checkpoint=1615000

In [ ]:
vocabulary_predict=load_vocabulary()

model.batch_size = 128

print(checkpoint)
model.predict(input_file="gs://bucket_context/eighth_experiment/testset/{}/inputs.txt".format(config), 
              output_file="gs://bucket_context/eighth_experiment/result_testset/predictions_{}.txt".format(config),
              checkpoint_steps=checkpoint, beam_size=1, temperature=0.0, keep_top_k=-1, vocabulary=vocabulary_predict)


INFO:root:system_path_file_exists:gs://bucket_context/eighth_experiment/ft_final_model_most_similar_crystalbleu/operative_config.gin
ERROR:root:Path not found: gs://bucket_context/eighth_experiment/ft_final_model_most_similar_crystalbleu/operative_config.gin


1615000


SimdMeshImpl ignoring devices ['', '', '', '', '', '', '', '']
Using default tf glorot_uniform_initializer for variable encoder/block_000/layer_000/SelfAttention/relative_attention_bias  The initialzer will guess the input and output dimensions  based on dimension order.
Using default tf glorot_uniform_initializer for variable decoder/block_000/layer_000/SelfAttention/relative_attention_bias  The initialzer will guess the input and output dimensions  based on dimension order.
Using default tf glorot_uniform_initializer for variable decoder/block_000/layer_000/SelfAttention/relative_attention_bias  The initialzer will guess the input and output dimensions  based on dimension order.
From /usr/local/lib/python3.8/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py:825: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
